From Stanford CS224N

# Word2Vec

The principle of word2vec is that it is possible to establish the meaning of each word based on the contexts in which it is used. Meanings are encoded as dense vectors.

Two words with similar vectors are expected to have similar meanings. 

## Two Algorithms: Skip-gram and Continuous Bag of Words
Skip-gram is concerned with predicting context words given a center word. Continuous Bag of Words (CBOW) is concerned with predicting the center word given context words.


## Terms

**Corpus**
A large body of continuous text.

**Vocabulary**
A vocabulary is a set of words in a corpus that vectors are calcuated for. Words that are rare in the corpus may not be included in the vocabulary.

**Center Word**
The word whose meaning is being computed.

**Context Word**
A word near the center word within a specified window.

**Window**
A number of word positions on either side of the center word. A window of size examines three words before the center word and three words to the right of the center word.



# Skip-gram Algorithm

## Principle
Go through each word position `t` in text. For the word at position `t` is a `center word`. Each word surrounding the center word is a `context word`. The algorithm considers words within a fixed distance of the center word. The fixed distance is a `window`. 

Let `c` be a center word and `o` be a context word or "outside" word.

Use the similarity of word vectors for `c` and `o` to calculate `P(o|c)`. This is the probability of a word in context given a center word. Adjust the word vectors to maximize the probability. Note that the CBOW algorithm is concerned with `P(c|o)`. 

### Note on Initializing Vectors
Vectors are initialized with random values. Initializing vectos with all zeros will yield poor results.

### Example: Window of Size 2

Let `w` be a word at the subscripted position. 

<font size=5>
$P(w_{t-2}|w_{t}) \; P(w_{t-1}|w_{t}) \;\;\;\;\;\;\;\;\;\;  P(w_{t+1}|w_{t}) \; P(w_{t+2}|w_{t}) $ <br>
$\;\;\; \text{problems} \;\; \text{turning} \;\;\;\;\; \text{into} \;\;\;\;\; \text{banking} \;\;\;\; \text{crisis}$<br>
$\;\;\; \text{window} \;\;\;\;  \text{window} \;\;\;\; \text{center} \;\; \text{window} \;\;\;\; \text{window}$<br>
</font>
<br>
<font size=5>
$ P(w_{t-2}|w_{t}) \; P(w_{t-1}|w_{t}) \;\;\;\;\;\;\;\;\;\;\;\;\;  P(w_{t+1}|w_{t}) \; P(w_{t+2}|w_{t})    $<br>
$\;\;\; \text{turning} \;\;\;\;\; \text{into} \;\;\;\;\;\;\;\;\; \text{banking} \;\; \text{crisis} \;\;\;\;\;\;\; \text{as}$<br>
$\;\;\; \text{window} \;\;\;\;  \text{window} \;\;\;\; \text{center} \;\;\;\; \text{window} \;\;\;\; \text{window}$<br>
</font>
The goal is to maximize the product of these probabilities.

## Skip-gram Likelihood

For each position `t = 1`, ..., `T`, predict context words within a window of fixed size `m`, given center word w[t]. The likelihood is a measure of how good the model is at predicting context words.

<font size=3>
$T = \text{number of words in a corpus} $ <br>
$t = \text{position of current word}  $ <br>
$m = \text{size of window, } \pm m \text{ words} $ <br>
$w = \text{word at a given position} $ <br>
$\theta = \text{contents of the vector, which will be updated by the algorithm} $ <br>
$L = \text{likelihood}$
</font>

For each word in the corpus, for each word in the window, get the probability that the context word is within the window of the given center word. This likelihood function 

<font size=5>
$$ L(\theta ) = \prod_{t = 1}^{T}\prod_{-m \leq j \leq m, \;j \ne 0}^{} P(w_{t+j}|w_{t};\theta) $$
</font>

## Skip-gram: Objective Function
The objective function is the average negative log likelihood. It is also called the **cost function** or **loss function**. We minimize the objective function in order to maximize predictive accuracy.

<font size=5>
$$ J(\theta ) = -\frac{1}{T}log  \; L(\theta) =  -\frac{1}{T} \sum_{t = 1}^{T}\sum_{-m \leq j \leq m, \;j \ne 0}^{} log \; P(w_{t+j}|w_{t};\theta) $$
</font>

For center word `c` and context word `o`

<font size=3>
$w = \text{a given word} $ <br>
$v_{w} = \text{vector of word when word is a center word} $ <br>
$u_{w} = \text{vector of word when word is a context word} $ <br>
$c = \text{center word}  $ <br>
$o = \text{context word}  $ <br>
$u_o^Tv_c = \text{dot product} = u_o \cdot v_c$ <br>
$u_w^Tv_c = \text{dot product} = u_w \cdot v_c$ <br>
</font>

## Probability of Context Word Given Center Word
The objective function depends on the ability to calculate the probability of a context word given a center word.

### The Probability
<font size=5>
$$ P(w_{t+j}|w_{t};\theta) $$<br>
</font>

### Approach
We will use two vectors per word $w$:
<font size=3>
*  $v_w$ when w is a center word
*  $u_w$ when w is a context word
</font>

### Example with `u` and `v` Notation

<font size=5>
$ P(u_{problems}|v_{into}) \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;P(u_{crisis}|v_{into})    $<br>
$ \;\;\;\;\;\;\;\;\;\;\; P(u_{turning}|v_{into}) \;\;\;\;\;\;\;  P(u_{banking}|v_{into}) \; $<br>
$\text{problems \;\; turning \;\;\;\;\; into \;\;\;\;\; banking \;\;\;\; crisis}$<br>
</font>

### Equation
Then for a center word `c` and context word `o`:

<font size=5>
$$ P(o|c) = \frac{exp(u_o^{\intercal}v_c)}{\sum_{w \in V} exp(u_w^{\intercal} v_c)} $$
</font>

### Characteristics of the Equation

<font size=3>
$exp$: Exponentiation makes anything positive.<br>
$u_o^{\intercal}v_c$: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Dot product compares similarity of `o` and `c`. It is the element-wise product.<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $u^{\intercal}v = u \cdot v = \sum_{i=1}^{n} u_iv_i$ <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Large dot product = larger probability<br>
$\sum_{w \in V} exp(u_w^{\intercal} v_c)$: Normalize over the entire vocabulary to give probability distribution.<br>
$w \in V$: If the word is a member of the vocabulary.
</font>

### Equation as Softmax

This is an example of the softmax function. The softmax function maps arbitrary values $x_{i}$ to a probability distribution $p_{i}$. It is called "max" because it amplifies the probability of the largest $x_{i}$ and "soft" because it still assigns some probability to smaller $x_{i}$.

<font size=5>
$softmax(x_i) =  \frac{exp(x_{i})}{\sum_{j=1}^{n} exp(x_{j})} =  p_{i}$
</font>

### Softmax in Code

In [1]:
import numpy as np
a = [1.0, 2.0, 3.0, 4.0, 1.0, 2.0, 3.0]
np.exp(a) / np.sum(np.exp(a)) 
#array([0.02364054, 0.06426166, 0.1746813, 0.474833, 0.02364054,
#       0.06426166, 0.1746813])

array([0.02364054, 0.06426166, 0.1746813 , 0.474833  , 0.02364054,
       0.06426166, 0.1746813 ])

## Training the Model: Compute `all` Vector Gradients
Recall $\theta$ represents all model parameters, in one long vector. **Remember that every word has two vectors**. In our case, with d-dimensional vectors and V-many words:

<font size=5>
$$\theta = 
\begin{bmatrix}
 v_{aardvark} \\ 
 v_{a} \\ 
 \vdots \\ 
 v_{zebra} \\ 
 u_{aardvark} \\ 
 u_{a} \\ 
 \vdots \\ 
 u_{zebra} \\ 
\end{bmatrix} = \in \mathbb{R}^{2dV}$$
</font>

## Minimizing the Objective Function

We need to minimize $J(\theta)$. Let's start with the derivative of $J(\theta)$ w.r.t $v_{c}$. Recall that $\theta$ is our parameters. The parameters are the contents of the $u$ and $v$ vectors.

<font size=5>
$$\frac{\partial J(\theta)}{\partial v_{c}} = \frac{\partial}{\partial v_{c}} log \frac{exp(u_{o}^{\intercal} v_{c})}{\sum_{w=1}^{V} exp(u_{o}^{\intercal}v_{c})} $$
</font>

Apply the log rule that $log \frac{a}{b} = log(a) - log(b)$

<font size=5>
$$= \frac{\partial}{\partial v_{c}}(log(exp(u_{o}^{\intercal}v_{c}))) - \frac{\partial}{\partial v_{c}}(log \sum_{w=1}^{V}exp(u_{o}^{\intercal}v_{c}))$$
</font>



## Left of the `-`

Recall that $log(exp(x)) = x$.



<font size=5>
$$\frac{\partial}{\partial v_{c}}(log(exp(u_{o}^{\intercal}v_{c}))) = \frac{\partial}{\partial v_{c}}(u_{o}^{\intercal}v_{c}) = u_{o}$$
</font>

Note that $v_{c}$ is a vector, meaning that this is multivariate calculus.

## Right of the `-`

### Review
First recall that.


<font size=3>
$\frac{d}{dx}ln(x) = \frac{1}{x}$<br>
$\frac{d}{dx}ln[f(x)] = \frac{1}{f'(x)}$<br>
$\frac{d}{dx}\sum x = \sum \frac{d}{dx}x$<br>
Chain rule: $\frac{d}{dx}f(g(x)) = f'g(x) * g'(x)$
</font>

Apply the chain rule and move the derivative inside the summation.

<font size=5>
$$\frac{\partial}{\partial v_{c}} = \frac{1}{\sum_{w=1}^{V}exp(u_{o}^{\intercal}v_{c})}\sum_{x=1}^{V}\frac{\partial}{\partial v_{c}}exp(u_{x}^{\intercal}v_{c})$$
</font>

Notice the "change of variable" to `x`. It is important use a different variable that we are summing over.

Apply the chain rule to the body of the inner summation.

<font size=5>
$$\sum_{x=1}^{V} \frac{exp(u_{x}^{\intercal}v_{c})}{\sum_{w=1}^{V}exp(u_{w}^{\intercal}v_{c})} * u_{x} $$
</font>

Notice that 

<font size=5>
$$  \frac{exp(u_{x}^{\intercal}v_{c})}{\sum_{w=1}^{V}exp(u_{w}^{\intercal}v_{c})} = P(x|c)$$
</font>

is the form of the probability distribution discussed in the section above softmax. Therefore, 

<font size=5>
$$ = \sum_{x=1}^{V} P(x|c) * u_{x}$$
</font>

## Combining the Two Sides of the `-`

<font size=5>
$$\frac{\partial}{\partial v_{c}} log P(o|c) = u_{o} - \sum_{x=1}^{V} P(x|c) * u_{x}$$
</font>

The meaning of this is that we take the observed representation of the context word $u_{o}$ and subtract from that the model thinks the context should look like. 

<font size=5>
$$\frac{\partial J(\theta)}{\partial v_{c}} = -u_{o} + \sum_{x=1}^{V} P(x|c) * u_{x}$$
</font>

## Now for the Context Words

The above was with respect to $v_{c}$, which means we did it for the center words. Now we need to do it for the context words, which us $u_{w}$.


`if w not equal 0`, w is not the context word.
<font size=5>$$ \frac{\partial J(\theta)}{\partial u_{w}} = \sum_{x=1}^{V} P(x|c) * v_{c}$$</font>

`if w equal 0`, w is the context word.
<font size=5>
$$ \frac{\partial J(\theta)}{\partial u_{w}} = -v + \sum_{x=1}^{V} P(x|c) * v_{c}$$
</font>

Once we have both the derivatives, we can use them in our SGD equation to update the weights.

However, there is one problem in this approach. As we can see, in the denominator, we have to take the exponential of the dot product of all our words and this is very time consuming when we have a huge vocabulary. We will need to train millions of weights which is not feasible.

So, in order to increase the training time, a new method was used called negative sampling. In this method, we will be updating only a small percentage of weights in one step. We will select a few -ve words i.e. the words which are not in the context window and we will change our weights in such a way that it maximizes the probability of real context words and minimize the probability of random words appearing around the center word. This change the loss function and now we are trying to maximize the following equation:
https://medium.com/analytics-vidhya/maths-behind-word2vec-explained-38d74f32726b

<font size=5>
$$J_{neg-sample}(o,v_{c},U) = -log(\sigma(u_{o}^{\intercal}
v_{c})) - \sum_{k=1}^{K} log(\sigma(-u_{k}^{\intercal} v_{c}))$$
</font>

where $K$ is the number of negative samples.

To select random words, we use unigram distribution where more frequent words are more likely to get selected.

To maximize the above term, we again need to take the derivative of the Loss function with respect to the weights, this case, it will be Uw, Uk, and Vc. Doing this in a similar way as we did above will give us the following three equations:

<font size=5>
$$ \frac{\partial J(\theta)}{\partial v_{c}} = - \sigma(-u_{o}^{\intercal}
v_{c})u_{o} + \sum_{k=1}^{K} \sigma(u_{k}^{\intercal} v_{c})u_{k}$$
</font>

<font size=5>
$$ \frac{\partial J(\theta)}{\partial u_{o}}  = - \sigma(-u_{o}^{\intercal}
v_{c})v_{c} $$
</font>

<font size=5>
$$ \frac{\partial J(\theta)}{\partial u_{k}} = \sum _{k=1}^{K} \sigma(u_{k}^{\intercal}v_{c})v_{c}$$
</font>

With all the derivatives calculated, now we can update our weight vectors little by little and get a vector representation that will point words appearing in a similar context in the same direction.

## Illustration

$\begin{bmatrix}
\bullet &\bullet  &\bullet  &\bullet &\bullet \\ 
\bullet &\bullet  &\bullet  &\bullet &\bullet \\ 
\bullet &\bullet  &\bullet  &\bullet &\bullet \\ 
\bullet &\bullet  &\bullet  &\bullet &\bullet \\ 
\bullet &\bullet  &\bullet  &\bullet &\bullet \\ 
\bullet &\bullet  &\bullet  &\bullet &\bullet  
\end{bmatrix} \; \;\;
\begin{bmatrix}
\bullet &\bullet  &\bullet  &\bullet &\bullet \\ 
\bullet &\bullet  &\bullet  &\bullet &\bullet \\ 
\bullet &\bullet  &\bullet  &\bullet &\bullet \\ 
\bullet &\bullet  &\bullet  &\bullet &\bullet \\ 
\bullet &\bullet  &\bullet  &\bullet &\bullet \\ 
\bullet &\bullet  &\bullet  &\bullet &\bullet  
\end{bmatrix} \; \;\; \;\;\;\;
\begin{bmatrix}
\bullet \\ 
\bullet \\ 
\bullet \\ 
\bullet \\ 
\bullet \\ 
\bullet 
\end{bmatrix} \; \;\; \;\;\;\;
\begin{bmatrix}
\bullet \\ 
\bullet \\ 
\bullet \\ 
\bullet \\ 
\bullet \\ 
\bullet 
\end{bmatrix}$ <br>
$\;\;\;\;\;\;\;\;\;\;\;\; U \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; V  \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; U\cdot v_{4}^{\intercal} \;\;\;\;\;\;\;\; softmax(U \cdot v_{4}^{\intercal})$<br>
$\;\;\;\;\;\;\;\;outside \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; center \;\;\;\;\;\;\;\;\;\; dot\;product \;\;\;\; probabilities $<br>

Same predictions at each position. One-to-the-left = "house", Two-to-the-left = "house", One-to-the-right = "house" if "house" is most probable context word.

We want a model that gives a reasonably high probability estimate to all words that occur in the context (fairly often).